In [36]:
import pandas as pd
import os
from main import get_miner
from log_parsing import from_id_to_template
import ast
from datetime import datetime

In [22]:
minscores = ['067']

In [23]:
df = pd.read_csv(f"output/features_labeled_v3_minscore_{minscores[0]}.csv")

In [24]:
df.head()

,abnormal_time,inject_pod,inject_time,inject_type,normal_time,result_list,root_cause
0,2022-08-22 03:54,frontend-579b9bff58-t2dbm,2022-08-22 03:53:54,cpu_contention,2022-08-22 03:51,"[{'events': '76_4', 'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'resource_alert': 'CpuUsageRate(%)', 'events_actual': '76_197'}]",['CpuUsageRate(%)']
1,2022-08-22 03:55,frontend-579b9bff58-t2dbm,2022-08-22 03:53:54,cpu_contention,2022-08-22 03:51,"[{'events': '76_4', 'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'resource_alert': 'CpuUsageRate(%)', 'events_actual': '76_197'}, {'events': '104_106', 'score': 0.7028169014084507, 'deepth': 2, 'pod': 'currencyservice-cf787dd48-vpjrd'}]",['CpuUsageRate(%)']
2,2022-08-22 04:03,frontend-579b9bff58-t2dbm,2022-08-22 04:02:07,return,2022-08-22 03:51,"[{'events': '120_198', 'score': 1.0, 'deepth': 4, 'pod': 'adservice-5f6585d649-fnmft', 'resource_alert': 'MemoryUsageRate(%)'}, {'events': '27_26', 'score': 1.0, 'deepth': 4, 'pod': 'adservice-5f6585d649-fnmft', 'resource_alert': 'MemoryUsageRate(%)'}, {'events': '28_119', 'score': 1.0, 'deepth': 4, 'pod': 'adservice-5f6585d649-fnmft', 'resource_alert': 'MemoryUsageRate(%)'}, {'events': '28_120', 'score': 1.0, 'deepth': 4, 'pod': 'adservice-5f6585d649-fnmft', 'resource_alert': 'MemoryUsageRate(%)'}, {'events': '18_78', 'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '18_191'}, {'events': '18_17', 'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '18_191'}, {'events': '18_80', 'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '18_191'}, {'events': '18_82', 'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '18_191'}, {'events': '18_84', 'score': 1.0, 'deepth': 1, ...","['Serving product page started', 'GetProduct start']"
3,2022-08-22 04:04,frontend-579b9bff58-t2dbm,2022-08-22 04:02:07,return,2022-08-22 03:51,"[{'events': '120_198', 'score': 1.0, 'deepth': 4, 'pod': 'adservice-5f6585d649-fnmft', 'resource_alert': 'MemoryUsageRate(%)'}, {'events': '27_26', 'score': 1.0, 'deepth': 4, 'pod': 'adservice-5f6585d649-fnmft', 'resource_alert': 'MemoryUsageRate(%)'}, {'events': '28_119', 'score': 1.0, 'deepth': 4, 'pod': 'adservice-5f6585d649-fnmft', 'resource_alert': 'MemoryUsageRate(%)'}, {'events': '28_120', 'score': 1.0, 'deepth': 4, 'pod': 'adservice-5f6585d649-fnmft', 'resource_alert': 'MemoryUsageRate(%)'}, {'events': '18_78', 'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '18_191'}, {'events': '18_17', 'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '18_191'}, {'events': '18_80', 'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '18_191'}, {'events': '18_82', 'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '18_191'}, {'events': '18_84', 'score': 1.0, 'deepth': 1, ...","['Serving product page started', 'GetProduct start']"
4,2022-08-22 04:11,frontend-579b9bff58-t2dbm,2022-08-22 04:10:20,cpu_consumed,2022-08-22 03:51,"[{'events': '76_4', 'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'resource_alert': 'CpuUsageRate(%)', 'events_actual': '76_197'}]",['CpuUsageRate(%)']


In [25]:
df['result_list'] = df['result_list'].apply(lambda x: ast.literal_eval(x))
df['root_cause'] = df['root_cause'].apply(lambda x: ast.literal_eval(x))

In [28]:
df['inject_type'].unique()

array(['cpu_contention', 'return', 'cpu_consumed', 'exception',
       'network_delay'], dtype=object)

In [30]:
df['root_cause'].apply(str).unique()

array(["['CpuUsageRate(%)']",
       "['Serving product page started', 'GetProduct start']",
       "['Placing order started', 'Order placed complete']",
       "['NetworkP90(ms)']",
       "['Start charge card', 'Charge successfully']",
       "['Received ad request', 'No context provided']",
       "['Query product with name', 'Query product successfully']"],
      dtype=object)

In [45]:
def get_timediff(inject_time, abnormal_time):
    inject_dt = datetime.strptime(inject_time, '%Y-%m-%d %H:%M:%S')
    abnormal_dt = datetime.strptime(abnormal_time, '%Y-%m-%d %H:%M')
    
    diff = max(1, abs(inject_dt.minute - abnormal_dt.minute))
    return diff

df["inject_comp_tdiff"] = df.apply(lambda row: get_timediff(row['inject_time'], row['abnormal_time']), axis=1)

In [46]:
df.inject_comp_tdiff.value_counts()

2    56
1    56
Name: inject_comp_tdiff, dtype: int64

In [31]:
log_template_miner = get_miner()

In [32]:
def get_nezha_rank(result_list, root_cause, inject_pod):
    rank = 1
    rc_found_flag = False
            
    if len(root_cause) == 1:
        for i in range(len(result_list)):
            if "resource_alert" in result_list[i].keys():
                if str(root_cause[0]) in str(result_list[i]["resource_alert"]) and inject_pod in str(result_list[i]["pod"]):
                    rc_found_flag = True
                    break
            else:
                if "events_actual" in result_list[i].keys() and str(root_cause[0]) in from_id_to_template(int(result_list[i]["events_actual"].split("_")[1]),log_template_miner) and inject_pod in str(result_list[i]["pod"]):
                    rc_found_flag = True
                    break
                if i > 0:
                    if result_list[i-1]["score"] == result_list[i]["score"] and result_list[i-1]["deepth"] == result_list[i]["deepth"]:
                        continue
                    else:
                        rank += 1
                elif i == 0:
                    rank += 1
    elif len(root_cause) == 2:
        for i in range(len(result_list)):
            if root_cause[0] in from_id_to_template(int(result_list[i]["events"].split(
                    "_")[0]),log_template_miner) and root_cause[1] in from_id_to_template(int(result_list[i]["events"].split("_")[1]),log_template_miner) and inject_pod in str(result_list[i]["pod"]):
                rc_found_flag = True
                break
            else:
                if i > 0:
                    # logger.info("%s, %s",
                    #             result_list[i-1]["score"], result_list[i]["score"])
                    if result_list[i-1]["score"] == result_list[i]["score"] and result_list[i-1]["deepth"] == result_list[i]["deepth"]:
                        continue
                    else:
                        rank += 1
                elif i == 0:
                    rank += 1
    
    if not rc_found_flag:
        return -1
    return rank

In [33]:
df["nezha_rank"] = df.apply(lambda row: get_nezha_rank(row['result_list'], row['root_cause'], row['inject_pod']), axis=1)

In [48]:
mask = df.inject_comp_tdiff == 2
df[mask].nezha_rank.value_counts()

 1    52
 2     2
-1     2
Name: nezha_rank, dtype: int64

In [ ]:
52/56, 54/56

(0.9285714285714286, 0.9642857142857143)

In [51]:
mask = df.inject_comp_tdiff == 1
df[mask].nezha_rank.value_counts()

 1    47
-1     7
 2     2
Name: nezha_rank, dtype: int64

In [52]:
47/56, 49/56

(0.8392857142857143, 0.875)

In [54]:
mask = df.nezha_rank == -1
pd.set_option('display.max_colwidth', 1000)
df[mask][['result_list', 'root_cause']]

,result_list,root_cause
18,[],"[Start charge card, Charge successfully]"
72,[],[CpuUsageRate(%)]
82,[],[NetworkP90(ms)]
83,"[{'events': '24_100', 'score': 0.7, 'deepth': 3, 'pod': 'currencyservice-cf787dd48-vpjrd'}]",[NetworkP90(ms)]
84,"[{'events': '198_120', 'score': 0.81, 'deepth': 4, 'pod': 'adservice-5f6585d649-fnmft', 'resource_alert': 'MemoryUsageRate(%)'}, {'events': '28_119', 'score': 0.81, 'deepth': 4, 'pod': 'adservice-5f6585d649-fnmft', 'resource_alert': 'MemoryUsageRate(%)'}, {'events': '18_17', 'score': 0.81, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '18_192'}, {'events': '18_80', 'score': 0.81, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '18_192'}, {'events': '18_82', 'score': 0.81, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '18_192'}, {'events': '18_84', 'score': 0.81, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '18_192'}, {'events': '18_86', 'score': 0.81, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '18_192'}, {'events': '16_15', 'score': 0.81, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm'}, {'events': '120_198', 'score': 0.7904040404040404, 'deepth': 4, 'pod': 'adservice-5f6585d649-fn...","[Query product with name, Query product successfully]"
88,"[{'events': '38_84', 'score': 0.9831081081081081, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '38_195'}, {'events': '38_37', 'score': 0.9805825242718447, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '38_195'}, {'events': '38_124', 'score': 0.9805825242718447, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '38_195'}, {'events': '198_120', 'score': 0.9759036144578314, 'deepth': 4, 'pod': 'adservice-5f6585d649-fnmft', 'resource_alert': 'MemoryUsageRate(%)'}, {'events': '28_119', 'score': 0.9759036144578314, 'deepth': 4, 'pod': 'adservice-5f6585d649-fnmft', 'resource_alert': 'MemoryUsageRate(%)'}, {'events': '18_17', 'score': 0.9759036144578314, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '18_192'}, {'events': '18_80', 'score': 0.9759036144578314, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '18_192'}, {'events': '18_82', 'score': 0.9759036144578314, 'deepth': 1, 'pod': 'frontend-57...","[Query product with name, Query product successfully]"
102,[],[NetworkP90(ms)]
103,"[{'events': '24_100', 'score': 0.7, 'deepth': 3, 'pod': 'currencyservice-cf787dd48-vpjrd'}, {'events': '23_100', 'score': 0.6744186046511628, 'deepth': 3, 'pod': 'currencyservice-cf787dd48-vpjrd'}]",[NetworkP90(ms)]
106,[],"[Received ad request, No context provided]"


In [55]:
def format_result_list(result_list):
    """
    Format result list for LLM comprehension
    """
    formatted_entries = [
        f"Ranked ambigous patterns (by comparing normal event traces with current faulty event traces)"
    ]
    for i in range(len(result_list)):
        entry = result_list[i]
        entry_details = [
            f"Pattern {1}:",
            f"- Expected Event Trace:",
            f"    From: {from_id_to_template(int(entry['events'].split('_')[0]), log_template_miner)}",
            f"    To: {from_id_to_template(int(entry['events'].split('_')[1]), log_template_miner)}",
        ]
        
        if 'events_actual' in entry:
            entry_details.extend([
            f"- Actual Event Trace:",
            f"    From: {from_id_to_template(int(entry['events_actual'].split('_')[0]), log_template_miner)}",
            f"    To: {from_id_to_template(int(entry['events_actual'].split('_')[1]), log_template_miner)}",
        ])
        
        entry_details.extend([
            
            f"- Abnormal Score: {entry.get('score', 'N/A')}"
            f"- Trace Depth: {entry.get('deepth', 'N/A')}"
            f"- Affected Pod: {entry.get('pod', 'N/A')}"
        ])
        
        if 'resource_alert' in entry:
            entry_details.append(f"- Resource Alert: {entry['resource_alert']}")
        
        formatted_entries.append("\n".join(entry_details))
    
    return "\n".join(formatted_entries)

In [56]:
df['result_list_string'] = df['result_list'].apply(format_result_list)

In [58]:
df[['result_list_string']].head()

,result_list_string
0,Ranked ambigous patterns (by comparing normal event traces with current faulty event traces)\nPattern 1:\n- Expected Event Trace:\n From: frontend hipstershop.Frontend/Recv. start\n To: TraceID: <:TRACEID:> SpanID: <:SPANID:> Request started\n- Actual Event Trace:\n From: frontend hipstershop.Frontend/Recv. start\n To: CpuUsageRate(%)\n- Abnormal Score: 1.0- Trace Depth: 1- Affected Pod: frontend-579b9bff58-t2dbm\n- Resource Alert: CpuUsageRate(%)
1,Ranked ambigous patterns (by comparing normal event traces with current faulty event traces)\nPattern 1:\n- Expected Event Trace:\n From: frontend hipstershop.Frontend/Recv. start\n To: TraceID: <:TRACEID:> SpanID: <:SPANID:> Request started\n- Actual Event Trace:\n From: frontend hipstershop.Frontend/Recv. start\n To: CpuUsageRate(%)\n- Abnormal Score: 1.0- Trace Depth: 1- Affected Pod: frontend-579b9bff58-t2dbm\n- Resource Alert: CpuUsageRate(%)\nPattern 1:\n- Expected Event Trace:\n From: currencyservice grpc.hipstershop.CurrencyService/Convert start\n To: currencyservice hipstershop.CurrencyService/Carry start\n- Abnormal Score: 0.7028169014084507- Trace Depth: 2- Affected Pod: currencyservice-cf787dd48-vpjrd
2,Ranked ambigous patterns (by comparing normal event traces with current faulty event traces)\nPattern 1:\n- Expected Event Trace:\n From: adservice hipstershop.AdService/<:SPANID:> start\n To: MemoryUsageRate(%)\n- Abnormal Score: 1.0- Trace Depth: 4- Affected Pod: adservice-5f6585d649-fnmft\n- Resource Alert: MemoryUsageRate(%)\nPattern 1:\n- Expected Event Trace:\n From: <:NUM:>:<:NUM:>:<:NUM:>.<:NUM:> INFO - TraceID: <:TRACEID:> SpanID: <:SPANID:> Query items successfully.\n To: <:NUM:>:<:NUM:>:<:NUM:>.<:NUM:> INFO - TraceID: <:TRACEID:> SpanID: <:SPANID:> Get Ads by Category <:*:>\n- Abnormal Score: 1.0- Trace Depth: 4- Affected Pod: adservice-5f6585d649-fnmft\n- Resource Alert: MemoryUsageRate(%)\nPattern 1:\n- Expected Event Trace:\n From: <:NUM:>:<:NUM:>:<:NUM:>.<:NUM:> INFO - TraceID: <:TRACEID:> SpanID: <:SPANID:> Received ad request (context words=<:ITEM:>\n To: adservice hipstershop.AdServiceImpl/getAds end\n- Abnormal Score: 1.0- Trace Depth: 4- Affect...
3,Ranked ambigous patterns (by comparing normal event traces with current faulty event traces)\nPattern 1:\n- Expected Event Trace:\n From: adservice hipstershop.AdService/<:SPANID:> start\n To: MemoryUsageRate(%)\n- Abnormal Score: 1.0- Trace Depth: 4- Affected Pod: adservice-5f6585d649-fnmft\n- Resource Alert: MemoryUsageRate(%)\nPattern 1:\n- Expected Event Trace:\n From: <:NUM:>:<:NUM:>:<:NUM:>.<:NUM:> INFO - TraceID: <:TRACEID:> SpanID: <:SPANID:> Query items successfully.\n To: <:NUM:>:<:NUM:>:<:NUM:>.<:NUM:> INFO - TraceID: <:TRACEID:> SpanID: <:SPANID:> Get Ads by Category <:*:>\n- Abnormal Score: 1.0- Trace Depth: 4- Affected Pod: adservice-5f6585d649-fnmft\n- Resource Alert: MemoryUsageRate(%)\nPattern 1:\n- Expected Event Trace:\n From: <:NUM:>:<:NUM:>:<:NUM:>.<:NUM:> INFO - TraceID: <:TRACEID:> SpanID: <:SPANID:> Received ad request (context words=<:ITEM:>\n To: adservice hipstershop.AdServiceImpl/getAds end\n- Abnormal Score: 1.0- Trace Depth: 4- Affect...
4,Ranked ambigous patterns (by comparing normal event traces with current faulty event traces)\nPattern 1:\n- Expected Event Trace:\n From: frontend hipstershop.Frontend/Recv. start\n To: TraceID: <:TRACEID:> SpanID: <:SPANID:> Request started\n- Actual Event Trace:\n From: frontend hipstershop.Frontend/Recv. start\n To: CpuUsageRate(%)\n- Abnormal Score: 1.0- Trace Depth: 1- Affected Pod: frontend-579b9bff58-t2dbm\n- Resource Alert: CpuUsageRate(%)


In [59]:
df.columns

Index(['abnormal_time', 'inject_pod', 'inject_time', 'inject_type',
       'normal_time', 'result_list', 'root_cause', 'nezha_rank',
       'inject_comp_tdiff', 'result_list_string'],
      dtype='object')

In [60]:
df.drop(columns="result_list").to_csv(f"output/features_labeled_4llm_v2_minscore_{minscores[0]}.csv", index=False)